In [17]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\catra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
import json
with open('songs.json', 'r') as json_file:
    data = json.load(json_file)

In [19]:
len(data)

13

In [20]:
data[0]

{'songName': 'Hurt',
 'spotifyLink': 'https://open.spotify.com/track/1lo9k4PrxFd5Np4cAxXoKo?si=881892db7eaa4a71',
 'genre': 'Rock',
 'lyrics': "I hurt myself today\nTo see if I still feel\nI focus on the pain\nThe only thing that's real\nThe needle tears a hole\nThe old familiar sting\nTry to kill it all away\nBut I remember everything\nWhat have I become?\nMy sweetest friend\nEveryone I know goes away\nIn the end\nAnd you could have it all\nMy empire of dirt\nI will let you down\nI will make you hurt\nI wear this crown of thorns\nUpon my liar's chair\nFull of broken thoughts\nI cannot repair\nBeneath the stains of time\nThe feelings disappear\nYou are someone else\nI'm still right here\nWhat have I become?\nMy sweetest friend\nEveryone I know goes away\nIn the end\nAnd you could have it all\nMy empire of dirt\nI will let you down\nI will make you hurt\nIf I could start again\nA million miles away\nI would keep myself\nI would find a way",
 'mood': '100stress'}

In [21]:
# convert prediction to value

def pred_to_mh(pred):
    label_mapping = {
        0: "Stressed",
        1: "Depressed",
        2: "Neutral",
        3: "Happy",
        4: "Studious"
    }

    return label_mapping[pred]

result = pred_to_mh(3)

In [22]:
result

'Happy'

In [23]:
word_list = {
    "Stressed": 
        ["stress", "uneasy", "nervous", "discomfort", "inconvenient", "Anxious", "Exhausting", "Intense", 
         "Panic", "Unmanageable", "Breakdown"],
    
    "Depressed":
        ["Melancholic", "Somber", "Blue", "Gloomy", "depression", "rain", "Overwhelming", "Desperate", 
         "Suicidal", "Paralyzed", "Incapacitating", "Unbearable", "Devastated", "cry", "tears", "pain"],
    
    "Neutral":
        ["bored", "simple", "pleasant", "normal", "neutral"],
        
    "Happy":
        ["Stable", "Calm", "Satisfied", "Happy", "Content", "Manic", "Euphoric", "Ecstatic", "Party",
         "Dance", "excited"],
        
    "Studious":
        ["smart", "study", "brain", "school", "grades", "graduation", "homework", "scholar", "work",
         "writing", "teacher", "students", "reading", "intelligent", "education"]
}

In [24]:
# clean (remove stopwords and whatnot)
import re
from nltk.corpus import stopwords

list_of_words = word_list[result]
list_of_song_lyrics = [song_record["lyrics"] for song_record in data]

stop_words = set(stopwords.words("english"))
stop_words.update({'im', 'em', 'on', 'say', 'cant', 'bout', 'about', 
                   'that', 'thats', 'put', 'askin', 'weve', 'got', 'could'
                   'would', 'let'})

def remove_stopwords(text):
    text_without_newlines = text.replace("\n", " ")
    words = text_without_newlines.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

def clean_all(list):    
    cleaned_lists = []
    for text in list:
        filtered_text_list = text.lower()
        filtered_text_list = re.sub(r'[^\w\s]', '', filtered_text_list)
        filtered_text_list = remove_stopwords(filtered_text_list)
        cleaned_lists.append(filtered_text_list)
    return cleaned_lists

list_of_song_lyrics = clean_all(list_of_song_lyrics)
list_of_song_lyrics

['hurt today see still feel focus pain thing real needle tears hole old familiar sting try kill away remember everything become sweetest friend everyone know goes away end could empire dirt make hurt wear crown thorns upon liars chair full broken thoughts cannot repair beneath stains time feelings disappear someone else still right become sweetest friend everyone know goes away end could empire dirt make hurt could start million miles away would keep would find way',
 'wasted times spent someone else wasnt even half reminiscin felt niscin felt even though life hell seem forget want belong give love pullin gettin sprung aint cause lot dont make run blowin spot cause aint business catchin feelings anyway aint business catchin feelings catchin feelings girls want youre winnin winnin youve beginnin ooh yeah yeah know right talkin talkin hope know dick still option cause ill beat ill beat yeah took time learn way body functions equestrian ride like champion ride baby sex get high without su

In [25]:
list_of_words = clean_all(list_of_words)
list_of_words[:5]

['stable', 'calm', 'satisfied', 'happy', 'content']

In [26]:
def build_corpus(list_of_words, list_of_song_lyrics):
    corpus = []
    
    for word in list_of_words:
        corpus.append(word)
    for song_lyrics in list_of_song_lyrics:
        for word in song_lyrics.split():
            corpus.append(word)
            
    corpus = list(set(corpus))
    return corpus

corpus = build_corpus(list_of_words, list_of_song_lyrics)
len(corpus)

596

In [27]:
import gensim
from gensim.models import word2vec

sentences = [word.split() for word in corpus]

model = word2vec.Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)

model.save("word2vec.model")

model.wv["war"]

array([-6.9636069e-03, -2.4585116e-03, -8.0229370e-03,  7.5005279e-03,
        6.1274157e-03,  5.2584694e-03,  8.3778575e-03, -6.9653272e-04,
       -9.3127284e-03,  9.1156662e-03, -4.9285362e-03,  7.8479899e-03,
        5.5338596e-03, -1.0790766e-03, -7.6642158e-03, -1.4598024e-03,
        6.2535368e-03, -6.9660828e-03,  1.4420962e-03, -7.9518585e-03,
        8.7213479e-03, -2.8557885e-03,  9.4373021e-03, -5.7080747e-03,
       -9.7177243e-03, -8.6279036e-03, -4.0748348e-03,  4.7095944e-03,
       -2.4193883e-04,  9.2235124e-03,  3.1092144e-03,  3.7477673e-03,
        2.9963492e-03,  8.1486488e-03, -2.3967146e-03,  7.4073388e-03,
       -9.5367134e-03,  2.9210865e-03, -6.8166968e-04,  4.5225740e-04,
        6.8430100e-03, -2.8419732e-03, -2.3567795e-03, -1.0047674e-04,
       -4.9769162e-04, -3.5749613e-03,  6.2444829e-03, -6.5586674e-03,
        7.8919996e-03, -9.3460083e-05,  2.6088404e-03,  3.2231498e-03,
       -2.8165340e-04,  1.7063022e-03, -3.1406546e-03,  4.7564553e-03,
      

In [28]:
sentences = [word.split() for word in list_of_words]
sentences[0]

['stable']

In [29]:
def word_embed(list, song = True):
    word_embeddings = []
    song_embeddings = []
    sentences = [word.split() for word in list]
    
    for sentence in sentences:
        word_embeddings.append(model.wv[sentence])
        
    if song:
         for sentence_embeddings in word_embeddings:
            song_embedding = np.mean(sentence_embeddings, axis=0)     
            song_embeddings.append(song_embedding) 
         return song_embeddings  
    
    else:
        return word_embeddings

In [30]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

list_of_words_embeddings = word_embed(list_of_words)
song_lyrics_embeddings = word_embed(list_of_song_lyrics)

list_avg_embedding = np.mean(song_lyrics_embeddings, axis=0)

similarities = [cosine_similarity([list_avg_embedding], [word_embedding])[0][0] for word_embedding in list_of_words_embeddings]

best_match_index = np.argmax(similarities)

best_matching_song = data[best_match_index]['songName']

best_matching_song

'Hurt'

In [31]:
similarities

[0.16187689,
 0.04182033,
 0.018520445,
 -0.028878463,
 -0.049142774,
 0.11702032,
 -0.13277635,
 0.09121998,
 -0.031889968,
 -0.016548418,
 0.030261124]

In [33]:
len(similarities)

11